In [1]:
import sys ,os
sys.path.append( "..")
from model.MTCformerV3 import MTCFormer
from torchinfo import summary
import torch.nn as nn

class WrappedModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return self.model(x, domain_lambda=0.0)[0]  # Only return task output for summary
    

model_former_curr = MTCFormer(
    depth=1,
    kernel_size=5,
    n_times=600,
    chs_num=7,
    eeg_ch_nums=4,
    class_num=2,
    class_num_domain=30,
    modulator_dropout=0.2,
    mid_dropout=0.5,
    output_dropout=0.5,
    weight_init_mean=0,
    weight_init_std=0.5,
    pooler_kernel_size=20,
    seed = 100,
)

model_former_curr = WrappedModel(model_former_curr)
summary(model_former_curr , input_size=[32 , 7 , 600])

Layer (type:depth-idx)                                  Output Shape              Param #
WrappedModel                                            [32, 2]                   --
├─MTCFormer: 1-1                                        [32, 2]                   --
│    └─TemporalModulator: 2-1                           [32, 4, 600]              --
│    │    └─Conv1d: 3-1                                 [32, 4, 600]              64
│    │    └─ChannelWiseLayerNorm: 3-2                   [32, 4, 600]              8
│    │    └─Dropout: 3-3                                [32, 4, 600]              --
│    │    └─Sigmoid: 3-4                                [32, 4, 600]              --
│    └─Conv1d: 2-2                                      [32, 8, 600]              40
│    └─ChannelWiseLayerNorm: 2-3                        [32, 8, 600]              16
│    └─GELU: 2-4                                        [32, 8, 600]              --
│    └─Dropout: 2-5                                     [32, 

In [2]:
bool((None,None))

True

In [4]:

model_former_curr = MTCFormer(
    depth=1,
    kernel_size=5,
    n_times=600,
    chs_num=7,
    eeg_ch_nums=4,
    class_num=2,
    class_num_domain=30,
    modulator_dropout=0.2,
    mid_dropout=0.5,
    output_dropout=0.5,
    weight_init_mean=0,
    weight_init_std=0.5,
    seed = 100,
)

model_former_curr

MTCFormer(
  (temporal_attention): TemporalModulator(
    (sensor_conv): Conv1d(3, 4, kernel_size=(5,), stride=(1,), padding=same)
    (sensor_norm): ChannelWiseLayerNorm((4,), eps=1e-05, elementwise_affine=True)
    (sensor_dropout): Dropout(p=0.2, inplace=False)
    (sigmoid): Sigmoid()
  )
  (patch_conv): Conv1d(4, 8, kernel_size=(1,), stride=(1,), padding=same)
  (patch_norm): ChannelWiseLayerNorm((8,), eps=1e-05, elementwise_affine=True)
  (patch_act): GELU(approximate='none')
  (patch_dropout): Dropout(p=0.5, inplace=False)
  (transformer): ConvolutionalAttention(
    (layers): ModuleList(
      (0): ConvolutionalAttentionBlock(
        (norm_1): LayerNorm((600,), eps=1e-05, elementwise_affine=True)
        (conv): Conv1d(8, 8, kernel_size=(5,), stride=(1,), padding=same)
        (activation_1): GELU(approximate='none')
        (dropout_1): Dropout(p=0.5, inplace=False)
        (norm_2): LayerNorm((600,), eps=1e-05, elementwise_affine=True)
        (feedforward): Linear(in_featur

In [6]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

# Example data: 10 signals, 8 channels, 1000 timepoints
X = np.random.randn(10, 8, 1000)

# Create sliding windows of size 128 with stride 1 (default)
window_size = 128
X_windowed = sliding_window_view(X, window_shape=window_size, axis=2)

# X_windowed.shape: (10, 8, 1000 - 128 + 1, 128)
print(X_windowed.shape)  # (10, 8, 873, 128)

(10, 8, 873, 128)


In [7]:
X_windowed = X_windowed[:, :, ::50, :]
X_windowed.shape

(10, 8, 18, 128)

In [ ]:
import torch

X = torch.randn(10, 8, 1000)  # [n_signals, channels, time]

In [20]:

window_size = 128
stride = 32

X_unfolded = X.unfold(dimension=2, size=window_size, step=stride).permute(0,2,1,3).reshape(-1 , 8 , 128)
# Shape: [10, 8, n_windows, 128]
print(X_unfolded.shape)

torch.Size([280, 8, 128])


In [17]:
X_unfolded[0]

tensor([[-1.0834, -1.5369, -0.5111,  ...,  0.1169,  0.8433, -0.2823],
        [ 0.5323, -1.0056,  0.0369,  ...,  0.9354,  0.7785,  0.7830],
        [-0.9183,  0.1634,  2.6049,  ..., -1.3368, -2.1676,  0.1507],
        ...,
        [-0.6156,  0.5981,  0.0175,  ...,  2.0049, -0.7799,  0.1091],
        [-1.0664,  1.8076, -0.5650,  ...,  0.0146, -0.3809, -0.9579],
        [-0.7578,  0.6202, -0.7500,  ..., -1.4003, -0.8852, -0.1490]])

In [21]:
X_unfolded[0]

tensor([[-1.0834, -1.5369, -0.5111,  ...,  0.1169,  0.8433, -0.2823],
        [ 0.5323, -1.0056,  0.0369,  ...,  0.9354,  0.7785,  0.7830],
        [-0.9183,  0.1634,  2.6049,  ..., -1.3368, -2.1676,  0.1507],
        ...,
        [-0.6156,  0.5981,  0.0175,  ...,  2.0049, -0.7799,  0.1091],
        [-1.0664,  1.8076, -0.5650,  ...,  0.0146, -0.3809, -0.9579],
        [-0.7578,  0.6202, -0.7500,  ..., -1.4003, -0.8852, -0.1490]])